In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts

#Additional import for time spectral connectivity
from mne_connectivity import spectral_connectivity_time

from mne_connectivity import spectral_connectivity_epochs
from mne.datasets import sample
import seaborn as sns

from scipy.signal import welch
import yasa
import constants
import numpy as np

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))


In [2]:
%%capture

data_type = 'N1'
folder = '/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/'
paths = joblib.load(folder + data_type + '_paths.pkl')
path = paths['selected_paths'][0]

method = 'pli'

channels = constants.channel_list
channel_names = constants.channel_list

data_epo = mne.read_epochs(path)
df_full = data_epo.to_data_frame()

#### Calculate connectivity over time

In [18]:
df_full.iloc[:7681]

,time,condition,epoch,Fp1,Fp2,F3,F4,C3,C4,P3,...,CP6,TP7,TP8,P1,P2,P5,P6,PO7,PO8,FCz
0,0.000000,N1,199,1.116644,-2.391638,-7.438055,0.128007,-0.280537,5.489225,2.318439,...,-2.863336,-2.517775,3.242273,4.666811,8.248322,5.066159,5.688397,-0.720770,1.512183,1.050704
1,0.003906,N1,199,-0.469489,-3.345754,-6.667561,1.268541,0.217403,7.992315,1.378157,...,-2.771452,-4.099472,1.256778,4.813881,8.901351,3.438273,5.361204,-1.716714,-0.197152,4.628359
2,0.007812,N1,199,0.253375,-2.825452,-3.605622,3.417794,1.524355,8.591318,-0.294488,...,-4.454839,-4.238951,-1.369752,4.031132,7.793636,0.441140,3.618188,-1.848537,-3.134845,8.199243
3,0.011719,N1,199,3.806369,0.007140,0.751531,6.025903,2.888049,7.052936,-2.380591,...,-7.454536,-3.857593,-4.222557,2.017343,5.002657,-3.047184,0.848377,-2.478843,-6.465273,10.743774
4,0.015625,N1,199,9.193107,4.919589,4.956121,8.596006,3.550612,4.012414,-4.276200,...,-10.610130,-3.851974,-6.953439,-0.766681,1.397495,-5.540364,-2.104871,-4.329993,-9.225382,11.613248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7676,29.984375,N1,199,-3.490251,-3.756713,-5.114113,-9.628959,-2.355396,-6.463572,-1.051495,...,5.448663,9.653478,16.921196,-3.966001,-0.694307,5.020816,12.293841,3.348407,15.503627,-15.460848
7677,29.988281,N1,199,-3.866194,-4.746308,-7.294515,-12.134253,-4.605698,-6.434278,-0.966377,...,7.627129,10.909268,18.976494,-4.039877,-1.133685,5.851084,14.318472,4.656179,17.820427,-18.217137
7678,29.992188,N1,199,-2.512540,-3.979311,-6.500729,-12.505222,-7.282581,-6.284318,-2.150823,...,8.582290,8.935105,19.711467,-5.065345,-2.304818,4.351438,14.753943,5.104577,19.206280,-19.762108
7679,29.996094,N1,199,0.602022,-1.571380,-2.901420,-10.440558,-9.208512,-6.158176,-4.630894,...,7.664149,5.265991,18.757904,-7.169618,-4.601559,0.594252,13.039687,3.827966,18.578738,-19.959638


In [19]:
all_data = df_full.iloc[:, 3:].values.T
all_data.shape
all_data = all_data.reshape(11,57,-1)
all_data.shape
all_data[0][0]

array([ 1.11664372, -0.46948885,  0.25337504, ..., -3.86619393,
       -2.51253972,  0.60202162])

In [66]:
# Something weird is happenning here, try grouping two epochs together at a time next!

In [53]:
connectivity_data_list = []
for epoch_val in np.unique(df_full.epoch.values):
    single_epoch_data = df_full[df_full['epoch'] == epoch_val].iloc[:, 3:].values
    single_epoch_data = single_epoch_data.T
    single_epoch_data = single_epoch_data.reshape(1,57,-1)
    con_pli =  spectral_connectivity_epochs(single_epoch_data, method=method , sfreq=256,fmin=16, fmax=30, faverage=True)
    connectivity_data = con_pli.get_data('dense')
    connectivity_data_list.append(connectivity_data)
    
# Create an empty DataFrame for the means
df = pd.DataFrame(index=channel_names, columns=channel_names)


# Fill the DataFrame with connectivity values
for i in range(len(channel_names)):
    for j in range(len(channel_names)):
        channel_1 = channel_names[i]
        channel_2 = channel_names[j]
        
        connectivity_values = []
        for connectivity_data in connectivity_data_list:
            connectivity_value = connectivity_data[i, j]
            connectivity_values.append(connectivity_value)
        connectivity_values = np.array(connectivity_values)
        df.loc[channel_1, channel_2] = connectivity_values
        df.loc[channel_2, channel_1] = connectivity_values

  
new_df_row = {}

for i, channel in enumerate(channels):
    for channel_2 in channels[i+1:]:
        val = df.loc[channel, channel_2]
        new_df_row['power_band' + '_' + channel + '_' + channel_2] = [val]
        
new_df_time = pd.DataFrame.from_dict(new_df_row, orient = 'columns')

Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 1596 connections
    using t=0.000s..29.996s for estimation (7680 points)
    frequencies: 16.0Hz..30.0Hz (421 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    assembling connectivity matrix
[Connectivity computation done]
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 1596 connections
    using t=0.000s..29.996s for estimation (7680 points)
    frequencies: 16.0Hz..30.0Hz (421 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    assembling connectivity matrix
[Connectivity computation done]
Connectivi

In [61]:
connectivity_value.mean()

0.0

In [65]:
new_df_time

,power_band_Fp1_Fpz,power_band_Fp1_Fp2,power_band_Fp1_F1,power_band_Fp1_Fz,power_band_Fp1_F2,power_band_Fp1_AF7,power_band_Fp1_F7,power_band_Fp1_F5,power_band_Fp1_F3,power_band_Fp1_AF8,...,power_band_PO8_POz,power_band_PO8_O1,power_band_PO8_O2,power_band_PO8_Oz,power_band_POz_O1,power_band_POz_O2,power_band_POz_Oz,power_band_O1_O2,power_band_O1_Oz,power_band_O2_Oz
0,"[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....",...,"[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1....","[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1...."


#### Normal way to calculated the connectivity

In [22]:
%%capture 

#generate coherence data across electrodes
# con_pli =  spectral_connectivity_epochs(data_epo , method=method , sfreq=256,fmin=16, fmax=30, faverage=True)
# con_pli =  spectral_connectivity_epochs(all_data, method=method , sfreq=256,fmin=16, fmax=30, faverage=True)

con_pli =  spectral_connectivity_epochs(data_epo, method=method , sfreq=256,fmin=16, fmax=30, faverage=True)

connectivity_data = con_pli.get_data('dense')
channel_data = connectivity_data

# Create an empty DataFrame for the means
df = pd.DataFrame(index=channel_names, columns=channel_names)

# Fill the DataFrame with connectivity values
for i in range(len(channel_names)):
    for j in range(len(channel_names)):
        channel_1 = channel_names[i]
        channel_2 = channel_names[j]
        connectivity_value = channel_data[i, j]
        df.loc[channel_1, channel_2] = connectivity_value.mean()
        df.loc[channel_2, channel_1] = connectivity_value.mean()

df = df.apply(pd.to_numeric)

new_df_row = {}

 #4. Go through all of the power_bands and add data as a column for a new dataframe with power_band + channel_1 + channel_2 as feature(s)
for i, channel in enumerate(channels):
    for channel_2 in channels[i+1:]:
        val = df.loc[channel, channel_2]
        new_df_row['power_band' + '_' + channel + '_' + channel_2] = [val]
        
new_df_2 = pd.DataFrame.from_dict(new_df_row, orient = 'columns')

In [27]:
(new_df_2 - new_df).max(axis = 1)

0    0.26517
dtype: float64

In [21]:
new_df

,power_band_Fp1_Fpz,power_band_Fp1_Fp2,power_band_Fp1_F1,power_band_Fp1_Fz,power_band_Fp1_F2,power_band_Fp1_AF7,power_band_Fp1_F7,power_band_Fp1_F5,power_band_Fp1_F3,power_band_Fp1_AF8,...,power_band_PO8_POz,power_band_PO8_O1,power_band_PO8_O2,power_band_PO8_Oz,power_band_POz_O1,power_band_POz_O2,power_band_POz_Oz,power_band_O1_O2,power_band_O1_Oz,power_band_O2_Oz
0,0.230404,0.223926,0.252861,0.24336,0.244224,0.267545,0.247247,0.258044,0.236018,0.224358,...,0.234723,0.224789,0.23645,0.214856,0.245951,0.223926,0.262794,0.24336,0.239473,0.233859


In [23]:
new_df_2

,power_band_Fp1_Fpz,power_band_Fp1_Fp2,power_band_Fp1_F1,power_band_Fp1_Fz,power_band_Fp1_F2,power_band_Fp1_AF7,power_band_Fp1_F7,power_band_Fp1_F5,power_band_Fp1_F3,power_band_Fp1_AF8,...,power_band_PO8_POz,power_band_PO8_O1,power_band_PO8_O2,power_band_PO8_Oz,power_band_POz_O1,power_band_POz_O2,power_band_POz_Oz,power_band_O1_O2,power_band_O1_Oz,power_band_O2_Oz
0,0.258044,0.468797,0.429497,0.346577,0.244656,0.363852,0.389765,0.372058,0.400561,0.361261,...,0.26884,0.28482,0.316346,0.264954,0.274887,0.267113,0.249838,0.292593,0.264954,0.239905


In [51]:
new_df

,power_band_Fp1_Fpz,power_band_Fp1_Fp2,power_band_Fp1_F1,power_band_Fp1_Fz,power_band_Fp1_F2,power_band_Fp1_AF7,power_band_Fp1_F7,power_band_Fp1_F5,power_band_Fp1_F3,power_band_Fp1_AF8,...,power_band_PO8_POz,power_band_PO8_O1,power_band_PO8_O2,power_band_PO8_Oz,power_band_POz_O1,power_band_POz_O2,power_band_POz_Oz,power_band_O1_O2,power_band_O1_Oz,power_band_O2_Oz
0,0.258044,0.468797,0.429497,0.346577,0.244656,0.363852,0.389765,0.372058,0.400561,0.361261,...,0.26884,0.28482,0.316346,0.264954,0.274887,0.267113,0.249838,0.292593,0.264954,0.239905
